### decompose

In [3]:
from prompt_manager import get_k_shot_with_answer, view_instruction, row_instruction
import pandas as pd
from utils import parse_specific_composition
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from sqlalchemy import create_engine
from executor import SQLManager
import sqlparse

In [4]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
                  "what was the time difference between the first place finisher and the eighth place finisher?",
                  # "compare the chart positions between the us and the uk for the science of selling yourself short, where did it do better?",
                  "other than william stuart price, which other businessman was born in tulsa?"
                # "what is the next most populous district after haridwar?",(70)
                #   "polona hercog 1890partner with alberta brianti after she have stephanie vogt as the partner",
                  ]
new_query_examples = [
  # "what was the chart position of 'The Science of Selling Yourself Short' in the US?; what was the chart position of 'The Science of Selling Yourself Short' in the UK?;",
                      "what was the time for the first place finisher?; what was the time for the eighth place finisher?",
                      "was william stuart price born in tulsa?; who was born in tulsa?"
                    # "what are the districts after haridwar?; what is the next most populous district after haridwar?",
                    #   "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      ]
num_k = 2
inds = [1, 11, 70, 42]
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]

examples = [TableFormat(format='none', data=normalised_data[i], use_sampling=True).format_nl_sep(normalised_data[i]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Query: {query}
Sub-Table: {table}
New query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt_wiki = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub-queries. Based on the table, provide at most 2 complete sub-queries for knowledge that you need. Output new query directly.""",
    suffix=
    """Query: {query}
Sub-Table: {table}
New query: """,
    input_variables=["query", "table"],
)



In [5]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
    # "after 2005 , the winner of the lifetime achievement award be andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle",
                  "all 12 club play a total of 22 game for the wru division one east",
                #   "a gamecube game loss the award in each of the first 3 year",
                "from 1980 to 2011 , apoel bc lose more than 2 time as many game as it win",
                  "polona hercog 1890partner with alberta brianti after she have stephanie vogt as the partner",
                  ]
task_examples = ["query rewrite", "query decompose", "query ambiguity resolve"]
new_query_examples = [
    # "Who were the winners of the lifetime achievement award after 2005?;",
                      "How many clubs play for the wru division one east in total?; How many clubs play 22 game for the wru division one east?;",
                    #   "a gamecube game loss the award in each of the first 3 year",
                    "from 1980 to 2011 , how many games did apoel bc lose?; from 1980 to 2011 , how many games did apoel bc win?;",
                      "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      ]
num_k = 3
inds = [1, 124, 5]
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Query: {query}
Sub-table: {table}
New query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub-queries. Based on the table, provide at most 2 sub-queries for knowledge that you need. Output new query directly.""",
    suffix=
    """Query: {query}
Sub-table: {table}
New query: """,
    input_variables=["query", "table"],
)

# Sub-questions are separated by semicolons.
# answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
#                                     template="""
# Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Query: {claim}
# answer the question given in the query. Only return the string instead of other format information. Do not repeat the question.
# """ )


In [6]:
task_name = 'wikitable'
split = 'test'
model_name = 'gpt-3.5-turbo-0125'
model = ChatOpenAI(model_name=model_name, openai_api_base="https://api.chatanywhere.com.cn/v1",
                       openai_api_key="sk-WZtqZEeuE0Xb6syVghDgAxdwe0ASWLkQRGxl61UI7B9RqNC4", temperature=0).bind(logprobs=True)
schema_information = pd.read_csv(f"result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine=engine)

In [14]:
table_loader = TableLoader(table_name=task_name, split='test', use_sample=True, small_test=False)
sample = table_loader.normalize_table(table_loader.dataset[2])

### step-back

In [8]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
inds = [8, 15,]
num_k = 2
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
examples = [TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_nl_sep(table_loader.dataset[inds[i]]['table']['caption']) for i in range(num_k)]
new_query_examples = [
    # "Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
    "which college list be public?",
    "which game be all score over time?"
    ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Query: {query}
Sub-Table: {table}
New query: {new_query}""")

examples_dict = [{"query": table_loader.dataset[inds[i]]['statement'],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Based on the table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """
Query: {query}
Sub-Table: {table}
New query:""",
    input_variables=["query", "table"],
)

In [40]:
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
i = 11
sample = table_loader.normalize_table(table_loader.dataset[i])
all_queries = []
formatter = TableFormat(format='none', data=sample, use_sampling=True)
# llm_chain = LLMChain(llm=model, prompt=decompose_prompt_wiki, verbose=True)
llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=True)
batch_pred = llm_chain.batch([{
    # "query": sample['query'],
    "query": "how many of the seasons games were played in the gold coast convention centre?",
                            #    "table": formatter.format_nl_sep()}],
                            "table": """table caption : 2008 - 09 nbl season.
col : date | home team | score | away team | venue | crowd | box score | report
row 1 : 31 december | cairns taipans | 105 - 112 | wollongong hawks | cairns convention centre | 3853 | box score | -
row 2 : 31 december | gold coast blaze | 103 - 94 | adelaide 36ers | gold coast convention centre | 2233 | box score | -"""}],)



> Entering new LLMChain chain...
Prompt after formatting:
Based on the table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.


Query: only 1 of the college list be public , and it be in new orleans
Sub-Table: gulf coast athletic conference
Col :institution|location|men_s_nickname|women_s_nickname|founded|type|enrollment|joined
Row 1 :edward waters college|jacksonville , florida|tigers|lady tigers|1866|private / (african methodist)|800|2010
Row 2 :talladega college|talladega , alabama|tornadoes|lady tornadoes|1867|private / (united church of christ)|600|1999 , 2011
Row 3 :dillard university|new orleans , louisiana|bleu devils|lady bleu devils|1869|private / (methodist & church of christ)|900|1981
New query: which college list be public?


Query: during the 1926 - 27 new york ranger season , game 26 , 29 , 30 , and 34 be all score in overtime
Sub-Table: 1926 - 27 new york rangers season
Col :game|february|opponent|sc

In [41]:
batch_pred[0]

{'query': 'how many of the seasons games were played in the gold coast convention centre?',
 'table': 'table caption : 2008 - 09 nbl season.\ncol : date | home team | score | away team | venue | crowd | box score | report\nrow 1 : 31 december | cairns taipans | 105 - 112 | wollongong hawks | cairns convention centre | 3853 | box score | -\nrow 2 : 31 december | gold coast blaze | 103 - 94 | adelaide 36ers | gold coast convention centre | 2233 | box score | -',
 'text': 'In which venue were the games played in the 2008-09 NBL season?'}

In [9]:
def get_k_shot_with_answer(k: int=1):
    sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"
            ]
    thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. 3 is the fewest points they received. "]
    tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
    claims = ["was 2 be the fewest point that roger dutton / tony wright receive?"]
    # inds from test split
    examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
    """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Query: {claim}
Thought: {thought}
Answer: {output}
    """)
    examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '3']))
    prompt_template = FewShotPromptTemplate(
        examples=[examples_dict],
        example_prompt=examples_prompt,
        prefix="""Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Think step by step and answer the last question given in the query.
You should output in the following format:
Thought: your step by step thought
Answer: Only return the concise string instead of other format information. Do not repeat the question.
Below is an example.""",
        suffix=
        """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Extra information:
{information}

Query: {query}""",
        input_variables=["table", "query", "SQL", "information"],
)
    return prompt_template

In [ ]:
[' who finish in first place and who do not manage to get in the top 3?', 'How many skaters finished in the top 3?', ' Who finished in first place?']

In [22]:
table_loader_wiki = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
TableFormat(format='none', data=table_loader_wiki.dataset[95], use_sampling=True).format_html(table_loader_wiki.dataset[95]['caption'])

'<table>\n<caption>Hoot Kloot</caption>\n<thead>\n<tr><th>  №</th><th>                      Title</th><th>  Directed_by_</th><th>  Released_</th></tr>\n</thead>\n<tbody>\n<tr><td>1  </td><td>"Kloot\'s Kounty"           </td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>2  </td><td>"Apache on the County Seat"</td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>6  </td><td>"Stirrups and Hiccups"     </td><td>Gerry Chiniquy</td><td>1973       </td></tr>\n</tbody>\n</table>'

In [25]:
# def get_k_shot_with_aug(k: int=2):
#     table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)

#     inds = [3, 6, 260, 33]
#     Output_examples = [
#                        'team, goals_for',
#                        'year, game, platform_s',
#                        'name, population_density_km_2_, population_2011_census_'
#                        'leading_scorer, score, date']
#     linking_examples = ['the team -> team; the most goal for -> goals_for',
#                         'gamecube -> platform_s; gamecube game -> game; the first 3 year -> year;',
#                         'alberta -> name; population density -> population_density_km_2_; 4257744 less people -> population_2011_census_; 2011 -> population_2011_census_'
#                         'jason richardson -> leading_scorer; leading scorer -> score; month -> date; 23 point per game -> leading_scorer'
#     ]
#     examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
#     """
#     Table: {table}
#     Query: {claim}
#     Column linking: {linking}
#     Columns: {output}""")
#     num_k = 3
#     examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_nl_sep(table_loader.dataset[inds[i]]['table']['caption']),
#                                         "claim": table_loader.dataset[inds[i]]['statement'],
#                                         "linking": linking_examples[i],
#                                         # "summary": summary_examples[i],
#                                         "output": Output_examples[i]} for i in range(num_k)]
#     prompt_template = FewShotPromptTemplate(
#         examples=examples_dict,
#         example_prompt=examples_prompt,
#         prefix=
#         """
#     Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the table.
#     Approach this task as follows:
#     Read the question thoroughly and list every possible link from query term to column in Table.
#     Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.""",
#     # You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
#     # Given the following table and query, you should output columns related to the query or contain useful information about the query. 
#     # Here are some examples:""",
#         suffix=
#         """
#     Table: {table}
#     Query: {claim}
#     Column linking:
#     """,
#         input_variables=["table", "claim"],
# )
#     return prompt_template

def get_k_shot_with_aug(k: int=2):
    table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
    table_loader_wiki = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
    inds = [3, 6, 260, 33]
    Output_examples = [
                       'team, goals_for',
                       'year, game, platform_s',
                       'name, population_density_km_2_, population_2011_census_'
                       'leading_scorer, score, date']
    linking_examples = ['the team -> team; the most goal for -> goals_for',
                        'gamecube -> platform_s; gamecube game -> game; the first 3 year -> year;',
                        'alberta -> name; population density -> population_density_km_2_; 4257744 less people -> population_2011_census_; 2011 -> population_2011_census_'
                        'jason richardson -> leading_scorer; month -> date; 23 point per game -> score'
    ]
    examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
    """
    Table: {table}
    Query: {claim}
    Column linking: {linking}
    Columns: {output}""")
    num_k = 2
    examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(table_loader.dataset[inds[i]]['table']['caption']),
                                        "claim": table_loader.dataset[inds[i]]['statement'],
                                        "linking": linking_examples[i],
                                        # "summary": summary_examples[i],
                                        "output": Output_examples[i]} for i in range(num_k)]
    examples_dict.extend([{"table": '<table>\n<caption>Hoot Kloot</caption>\n<thead>\n<tr><th> Number</th><th> Title</th><th> Directed_by_</th><th> Released_</th></tr>\n</thead>\n<tbody>\n<tr><td>1  </td><td>"Kloot\'s Kounty"           </td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>2  </td><td>"Apache on the County Seat"</td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>6  </td><td>"Stirrups and Hiccups"     </td><td>Gerry Chiniquy</td><td>1973       </td></tr>\n</tbody>\n</table>',
                                        "claim": table_loader_wiki.dataset[95]['question'],
                                        "linking": "the last title -> Released_ the last title-> Number; title -> Title",
                                        # "summary": summary_examples[i],
                                        "output": "Title, Released_, Number"}])
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
    Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the table.
    Approach this task as follows，read the claim thoroughly and list every possible link from query term to column in Table. Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.""",
    # You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
    # Given the following table and query, you should output columns related to the query or contain useful information about the query. 
    # Here are some examples:""",
        suffix=
        """
    Table: {table}
    Extra information: {aug}
    Query: {claim}""",
        input_variables=["table", "claim", "aug"],
)
    return prompt_template


In [26]:
from utils import parse_output
sample = table_loader.normalize_table(table_loader.dataset[200])
k_shot_prompt = get_k_shot_with_aug()
formatter = TableFormat(format='none', data=sample, use_sampling=True)
llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_aug(), verbose=True)
summary_information = pd.read_csv(f"result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
schema_information = pd.read_csv(f"result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
composition_information = pd.read_csv(f"result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
summary_aug, column_aug = summary_information.loc[sample['table']['id']]['summary'], summary_information.loc[sample['table']['id']]['column_description'] 
col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
extra_col_info = []
for i_c in range(len(col_names)):
    extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
stage_1_batch_pred = llm_chain.batch([dict({
    'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': sample['query'],
                                    # 'claim': sample['query'],
                                    'aug': ""
                                    # 'aug':  summary_aug + '\n'.join(extra_col_info),
                                    })], return_only_outputs=True)[0]['text']
# pred = llm_chain.batch([dict({"query": 'Who is one of the three nominees for a Drama Desk Award?'})])
# print(pred[0]['text'])
print(stage_1_batch_pred)




> Entering new LLMChain chain...
Prompt after formatting:

    Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the table.
    Approach this task as follows，read the claim thoroughly and list every possible link from query term to column in Table. Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.


    Table: <table>
<caption>1986 - 87 north west counties football league</caption>
<thead>
<tr><th>  position</th><th>               team</th><th>  played</th><th>  drawn</th><th>  lost</th><th>  goals_for</th><th>  goals_against</th><th>  goal_difference</th><th>  points_1</th></tr>
</thead>
<tbody>
<tr><td>12        </td><td>ashton town        </td><td>24      </td><td>9      </td><td>11    </td><td>29         </td><td>39             </td><

In [ ]:
from utils import parse_output
def scene_A(query, sample, verbose=True):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        if pd.isna(summary_aug):
            summary_aug = ''
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]
        formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
        try: 
            formatter.data = formatter.data.loc[:, columns]
            # formatter.all_data = formatter.all_data.loc[:, columns]
        except:
            pass
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + extra_information
                                            })], return_only_outputs=True)[0]['text']
    print("total_tokens:", cb.total_tokens)
    print(stage_2_batch_pred)
    # stage 3: SQL Excution
    try: 
        formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
    except:
        formatter.data = formatter.all_data
        stage_2_batch_pred = 'SELECT * from DF;'
    if len(formatter.data) == 0:
        return query, stage_2_batch_pred, 'No data from database', cb.total_tokens
    return query, stage_2_batch_pred, formatter.format_html(), cb.total_tokens

In [12]:
inds = [0,   1,  11,  14,  22,  24,  26,  31,  37,  38,  39,  40,  47,
        49,  50,  59,  62,  63,  65,  66,  68,  69,  70,  75,  80,  81,
        82,  90,  91,  93,  95,  99, 100, 105, 106, 108, 110, 116, 119,
       124, 129, 136, 137, 145, 147, 148, 151, 153, 155, 158, 160, 164,
       167, 169, 171, 173, 175, 179, 180, 181, 187, 188, 190, 192, 198,
       200, 207, 209, 210, 211, 217, 225, 227, 230, 233, 236, 237, 241,
       251]

In [10]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

In [12]:
from typing import List
import os
import json
def save_csv(input_list: List[List], label_list: List, file_path):
    import pandas as pd
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    assert len(input_list) == len(label_list)
    df = pd.DataFrame()
    for i in range(len(label_list)):
        df[label_list[i]] = pd.Series(input_list[i])
    if os.path.exists(file_path) and file_path.endswith('.csv'):
        df_origin = pd.read_csv(file_path)
        df = pd.concat([df_origin, df], axis=0)
    df.to_csv(file_path, index=False, encoding='utf-8')

### RUN

In [67]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.com.cn/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)

save_path = f"result/answer/tabfact_zh_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Below are some sub-tables, each sub-table is generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task by using useful information below.
You can use one of the sub-tables or use all sub-tables.
{information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
tokens = []
outputs = []
labels = []
stage_1 = []
stage_2 = []
# muilti_answer_instruction = get_k_shot_with_answer()
i = 
    
sample = table_loader.normalize_table(
                    table_loader.dataset[i])
labels.append(sample['label'])
all_queries = [sample['query']]
formatter = TableFormat(format='none', data=sample, use_sampling=True)
llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)
all_queries.append(batch_pred[0]['text'].split(':')[-1])
print(batch_pred[0]['text'].split(':')[-1])

args_list = [{"query": q, "sample": sample} for q in all_queries]
results = parallel_run_kwargs(scene_A, args_list)
temp = [f"""
SQL Excuted: 
```{res[1]}```
Sub-table: {res[2]}""" for res in results if len(res[1]) > 0]
tokens.append(sum([res[3] for res in results]) / len(results))
llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
batch_pred = llm_chain.batch([{"query": sample['query'], "information": '\n'.join(temp)}], return_only_outputs=True)
print(batch_pred[0])
outputs.append(batch_pred[0]['text'])
# llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
# batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html(table_caption=sample['table']['caption'])}], return_only_outputs=True)
# print(batch_pred[0]['text'].split(':')[-1].split(';'))
# all_queries.extend(batch_pred[0]['text'].split(':')[-1].split(';'))
# "Is the following query true or false?" +

SyntaxError: invalid syntax (1272733091.py, line 23)

In [9]:
acc = 0
non_equal = []
for ind, (pred, gold) in enumerate(zip(ttt, labels )):
            if pred == str(gold):
                acc += 1
            else:
                non_equal.append(ind)

In [11]:
print(non_equal)

[1, 8, 11, 14, 22, 26, 29, 37, 38, 40, 47, 49, 50, 56, 57, 59, 63, 65, 66, 68, 72, 76, 78, 82, 84, 87, 88, 90, 95, 96, 99]


In [8]:
[0, 1, 3, 14, 22, 38, 40, 42, 43, 44, 47, 49, 50, 51, 53, 59, 61, 63, 68, 69, 72, 74, 75, 76, 78, 80, 82, 83, 89, 91, 92, 98, 99]

[0, 1, 3, 14, 22, 25, 33, 38, 40, 42, 43, 44, 47, 49, 50, 51, 53, 59, 61, 63, 68, 69, 72, 74, 75, 76, 78, 80, 82, 83, 89, 91, 92, 98, 99]


In [13]:

1, 3, 11, 12, 13, 14, 16, 20, 22, 26, 34, 38, 
40, 41, 42, 44, 47, 50, 53, 56, 64, 65, 69, 72,
79, 80, 82, 83, 84, 88, 90, 91, 92, 95, 98, 100,
108, 110, 113, 116, 119, 121, 122, 123, 125, 133, 
136, 137, 139, 140, 141, 144, 145, 148, 158, 167,
169, 174, 176, 177, 179, 181, 185, 191, 200, 201, 210, 
220, 223, 225, 226, 227, 228, 229, 231, 238, 240, 241, 
246, 251, 254, 255, 256, 265, 269, 270, 272, 274, 280, 
281, 282, 283, 286, 287, 294, 296

[1, 3, 11, 12, 13, 14, 16, 20, 22, 26, 34, 38, 40, 41, 42, 44, 47, 50, 53, 56, 64, 65, 69, 72, 79, 80, 82, 83, 84, 88, 90, 91, 92, 95, 98, 100, 108, 110, 113, 116, 119, 121, 122, 123, 125, 133, 136, 137, 139, 140, 141, 144, 145, 148, 158, 167, 169, 174, 176, 177, 179, 181, 185, 191, 200, 201, 210, 220, 223, 225, 226, 227, 228, 229, 231, 238, 240, 241, 246, 251, 254, 255, 256, 265, 269, 270, 272, 274, 280, 281, 282, 283, 286, 287, 294, 296]


In [30]:
print(data.iloc[25, :]['preds'])

To verify the claim, we need to look at the sub-table and check if there are at least 3 people who tie for fifth place and are from the United States.

Since we do not have access to the sub-table, we cannot directly verify the claim. However, based on the SQL query provided, we can infer that the query is filtering for players who are in fifth place and from the United States. Therefore, if the query returns at least 3 players, then the claim is true.

So, based on the logic of the SQL query, if the query returns at least 3 players, the claim is true. If not, the claim is false.

Therefore, we cannot definitively determine the truth of the claim without the sub-table.


In [27]:
data.head()

,table_name,extra_information,preds,token
0,1-29063233-1.html.csv,"The episode ""The Nightmare Begins"" happened ea...","To verify the claim that the episode ""Sweet Dr...",4077
1,1-29063233-1.html.csv,NaN,"To verify the claim that ""David Moore directed...",4338
2,1-29063233-1.html.csv,"The episode of ""The Lady of the Lake"" with the...",The SQL query is filtering the table DF to onl...,4469
3,1-29063233-1.html.csv,Cannot get answer from sub-table.,The SQL query is filtering the table DF to onl...,3912
4,1-29063233-1.html.csv,Beauty and the Beast (part 2) has more UK view...,The SQL query retrieves the titles and UK view...,4182


In [7]:
import  pandas as pd
data = pd.read_csv('./result/answer/tabfact_zh_04-25_15-42-17.csv')

ttt = eval_blury_string(data['preds'])
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
labels = []
for i in range(100):
    labels.append(table_loader.dataset[i]['label'])

In [19]:
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [20]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)
# args_list = [{"query": 'Who took the loss in the game on August 30 and who suffered the loss in the game on August 31?', "sample": sample},{"query": sample['query'], "sample": sample}]
# args_list = [{"query": q, "sample": sample} for q in all_queries]

# results = parallel_run_kwargs(scene_A, args_list)
# print(results)
# print(cb.total_tokens)


In [ ]:
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
                                    template="""
Below are some sub-tables generated by excuting the SQL. You need to understand the logic behind the SQL filtering. Complete task using all information below. 
{information}
Query: {query}
verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step.
""" )
temp = [f"""
SQL Excuted: 
```{res[1]}```
Sub-table: {res[2]}""" for res in results]
muilti_answer_instruction = get_k_shot_with_answer()
llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
batch_pred = llm_chain.batch([{"query": sample['query'], "information": '\n'.join(temp)}], return_only_outputs=True)

In [35]:
def scene_B():
    agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Based on the history information, your task is to only based on the conversation information to answer the user query.
    If you cannot get the answer from past history, reorganize the question and return the question explicitly. If you are confident in the answer, answer it directly.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
)
    chain = LLMChain(llm=model, prompt=agent_prompt, verbose=True)
    return chain.invoke(
    {
        "chat_history": Agent_history.messages,
    }
)['text']
    
    #维护一个Agent Memory
Agent_history = []
agent_prompt = ChatPromptTemplate.from_messages(
    [
        # (
        #     "system",
        #     """"""
            
        #     # return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.,
        # ),
        MessagesPlaceholder(variable_name="chat_history"),
    ]
)
system =  PromptTemplate(input_variables=["history"], template="""Based on the history information, your task is to only based on the conversation information to answer the user query. 
Note: Do not use information on your own, only use information from the conversation history!

conversation histroy:
{history}

The output should choose from Choice A and Choice B:
Choice A: ###If you cannot get the answer from conversation histroy, reorganize the question and return the question explicitly.
Choice B: ###If you are confident in the answer, answer it directly.""")
chain = LLMChain(llm=model, prompt=system, verbose=True)
Agent_history.append('Q: Who were the winners of the lifetime achievement award after 2005?')
# Agent_history.append('A: the winners are andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle')
# Agent_history.append('Q: Are the winners andrew rule john silvester , sandra harvey lindsay simpson , marele day , shane maloney , and peter doyle?')


In [ ]:
import concurrent.futures
 
def outer_task(url):
   # 外层任务
   print(f"Processing {url}")
   # 内部再次使用线程池
   with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
       # 执行一些依赖于外层任务的结果的并行操作
       inner_results = [executor.submit(inner_task, f"{url}_{i}") for i in range(2)]
       # 等待内部任务完成并收集结果
       return [r.result() for r in inner_results]
 
def inner_task(url):
   # 内层任务
   print(f"Inner task for {url}")
   # 这里可以执行一些操作，比如I/O密集型的任务
   return f"Result for {url}"
 
if __name__ == "__main__":
   # 创建线程池
   with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
       # 提交外层任务到线程池
       outer_results = [executor.submit(outer_task, f"http://example.com/{i}") for i in range(2)]
       # 等待外层任务完成并收集结果
       for future in concurrent.futures.as_completed(outer_results):
           print(future.result())

## 调整extrainformation的位置

In [11]:
from utils import parse_output
answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
                                    template="""
Below is a sub-table generated by excuting the SQL. You need to understand the logic behind the SQL filtering and complete task using the final sub-table. 
SQL Excuted: 
```{SQL}```
Sub-table: 
{table}
Query: {claim}
Please provide a clear, concise statement in response to the question. If you cannot answer the question based on the sub-table, just say 'Cannot get answer from sub-table'
""" )
def scene_B(query, sample, verbose=False):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
    formatter = TableFormat(format='none', data=sample, use_sampling=True)
    k_shot_prompt = get_k_shot_with_aug()
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        if pd.isna(summary_aug):
            summary_aug = ''
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = [formatter.normalize_col_name(c.strip()) for c in stage_1_batch_pred.split(',')]
        
        try: 
            formatter.data = formatter.data.loc[:, columns]
        except:
            pass
        formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
        extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], formatter.data.columns))
        stage_2_batch_pred = llm_chain.batch([dict({'table': formatter.format_html(table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + extra_information
                                            })], return_only_outputs=True)[0]['text']
    
        
        print(stage_2_batch_pred)
        # stage 3: SQL Excution
        try: 
            formatter.data = manager.execute_from_df(stage_2_batch_pred, formatter.all_data, table_name='DF')
        except:
            formatter.data = formatter.all_data
            stage_2_batch_pred = 'SELECT * from DF;'
        llm_chain = LLMChain(llm=model, prompt=answer_instruction, verbose=verbose)
        response = llm_chain.batch([dict({'table': formatter.format_html(),
                                                'claim': query,
                                                'SQL':  stage_2_batch_pred
                                                })], return_only_outputs=True)[0]['text']
    # print("total_tokens:", cb.total_tokens)
    return response, cb.total_tokens

In [12]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = "0c75de50975e4f278b882fe90da47f2f"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ces.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)


In [44]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
from FlagEmbedding import FlagReranker
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=True, small_test=False)
# model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.com.cn/v1",
#                        openai_api_key="sk-WZtqZEeuE0Xb6syVghDgAxdwe0ASWLkQRGxl61UI7B9RqNC4", temperature=0.01)
save_path = f"result/answer/wikitable_zh_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)
muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
# template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
template = """
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.

SQL Excuted: 
```{SQL}```
Sub-table:
{table}
Extra information:
{information}

Query: {query}
Think step by step and answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
""" )
# Task: answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
# Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
tokens = []
outputs = []
labels = []
ids = []
# muilti_answer_instruction = get_k_shot_with_answer()
i = 200
while i < 300:
    try:
        sample = table_loader.normalize_table(
                            table_loader.dataset[i])
        all_tokens = 0
        all_queries = []
        formatter = TableFormat(format='none', data=sample, use_sampling=True)
        with get_openai_callback() as cb:
            llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
            batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)
            all_queries.append(batch_pred[0]['text'].strip())
            llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
            batch_pred = llm_chain.batch([{"query": sample['query'], "table": formatter.format_html()}], return_only_outputs=True)
            all_queries.extend(batch_pred[0]['text'].split(';'))
            print(all_queries)
        all_tokens += cb.total_tokens
        args_list = [{"query": q, "sample": sample} for q in all_queries if reranker.compute_score([(q, sample['query'])], normalize=True) < 0.95]
        ans_from_B = parallel_run_kwargs(scene_B, args_list)
        results = [res[0] for res in ans_from_B if res[0] != 'Cannot get answer from sub-table']
        all_tokens += sum([res[1] for res in ans_from_B])
        #With answer
        with get_openai_callback() as cb:
            imp_input = scene_A(sample['query'], sample, True)
            llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_answer(), verbose=True)
            batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
        print(batch_pred[0])
        all_tokens += cb.total_tokens
        print('ALL TOKENS', all_tokens)
        ids.append(sample['id'])
        labels.append(sample['query'])
        outputs.append(batch_pred[0]['text'])
        i += 1
        if (i + 1) % 10 == 0:
            save_csv([outputs, labels, ids], ['preds', 'statements','ids'], file_path=save_path)
            outputs = []
            labels = []
            ids = []
    except:
        pass
#With no answer
# temp = [f"""
# SQL Excuted for extra information: 
# ```{res[1]}```
# Sub-table for extra information: {res[2]}""" for res in results if res[2] != 'No data from database']
# imp_input = scene_A(sample['query'], sample, False)
# llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
# batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(temp)}], return_only_outputs=True)
# print(batch_pred[0])
# outputs.append(batch_pred[0]['text'])


----------using 4*GPUs----------
['what is the name of the last episode?', 'What is the name of the last episode title?', ' What is the premiere date of the last episode?']
Columns: Premiere_Date
SELECT Premiere_Date
FROM DF
ORDER BY Premiere_Date DESC
LIMIT 1;


> Entering new LLMChain chain...
Prompt after formatting:

    Your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the table.
    Approach this task as follows，read the claim thoroughly and list every possible link from query term to column in Table. Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.


    Table: <table>
<caption>1986 - 87 north west counties football league</caption>
<thead>
<tr><th>  position</th><th>               team</th><th>  played</th><th>  drawn</th><th>  los

In [43]:
save_path

'result/answer/wikitable_zh_05-04_07-27-00.csv'

In [106]:
duplicates_count = pd.Series(formatter.data.columns, index=formatter.data.columns).groupby(formatter.data.columns).cumcount()
rename_list = [f"{col_name}_{col_count}" if col_count > 0 else col_name for (col_name, col_count) in duplicates_count.items()]

In [107]:
rename_list

['Year', 'Name', 'Year_1', 'Name_1', 'Year_2', 'Name_2']

In [101]:
rename_dict

{'Year': 'Year_2', 'Name': 'Name_2'}

In [83]:
rename_dict = {col: f"{col}_{i}" if duplicates_count[col] > 1 else col for i, is_rep in formatter.data.columns.duplicated()}

AttributeError: 'PrettyDict' object has no attribute 'cumcount'

In [15]:
len([1, 11, 14, 22, 26, 37, 38, 40, 47, 49, 50,  59, 63, 65, 66, 68, 82, 88, 99])

20

In [ ]:
46: west berkshire brewery 's maggs magnificent mild be its most decorate beer between 1995 and 2009

In [29]:
20 ：Based on the information provided, the SQL query filtered for records where the place starts with 't9' and the country is 'united states'. The result of the query shows that there are 3 people who meet these criteria. However, the extra information states that there were actually 4 people who tied for ninth place, and all of them were from the United States.\n\nTherefore, the provided claim/query is false. The correct number of people from the United States who tied for ninth place is 4, not 3. \n\nFinal answer: 0
88: To verify whether the term start for Bashkim Fino is after the term start for Vilson Ahmeti, we need to compare the dates mentioned in the sub-table. From the sub-table:\n- Vilson Ahmeti's term started on December 10, 1991.\n- Bashkim Fino's term started on March 11, 1997.\n\nSince December comes before March in the calendar year, it is evident that Vilson Ahmeti's term started before Bashkim Fino's term. Therefore, the claim that the term start for Bashkim Fino is after the term start for Vilson Ahmeti is FALSE.\n\nHence, the answer is 0
72:To verify the claim that the gap between the first and last player being a total of 58.04 is true, we need to calculate the difference between the points of the first and last player based on the given information.\n\nGiven:\n- Total gap between the first and last rank is 18.\n- Rank of the first player is 1.\n- Rank of the last player is 19.\n\nLet's calculate the points difference between the first and last player:\npoints difference = (points of last player) - (points of first player)\n\nSince the total gap between the first and last rank is 18, we can calculate the points difference as follows:\npoints difference = 18 * (MAX(points) - MIN(points))\n\nGiven that the calculated point gap from the sub-table is 58.04, we can substitute this value into the formula:\n18 * 58.04 = 1044.72\n\nTherefore, the claim that the gap between the first and last player being a total of 58.04 is false.\n\nFinal answer: 0
76:The SQL query filters for competitors from France with a rank less than 5. Since the sub-table generated from the query shows no data, it means that there were no competitors from France who finished better than 5th place. \n\nTherefore, the provided claim that France's competitors all finished better than 5th place is TRUE.\n\nFinal answer: 1

SyntaxError: invalid character '：' (U+FF1A) (3998830562.py, line 1)

In [ ]:
1, 3, 11, 12, 13, 14, 16, 20, 22, 26, 34, 38, 
40, 41, 42, 44, 47, 50, 53, 56, 64, 65, 69, 72,
79, 80, 82, 83, 84, 88, 90, 91, 92, 95, 98, 100,
108, 110, 113, 116, 119, 121, 122, 123, 125, 133, 
136, 137, 139, 140, 141, 144, 145, 148, 158, 167,
169, 174, 176, 177, 179, 181, 185, 191, 200, 201, 210, 
220, 223, 225, 226, 227, 228, 229, 231, 238, 240, 241, 
246, 251, 254, 255, 256, 265, 269, 270, 272, 274, 280, 
281, 282, 283, 286, 287, 294, 296

In [30]:
print(data.iloc[88, :]['preds'])

To verify whether the term start for Bashkim Fino is after the term start for Vilson Ahmeti, we need to compare the dates mentioned in the sub-table.

From the sub-table:
- Vilson Ahmeti's term started on 10th December 1991.
- Bashkim Fino's term started on 11th March 1997.

Since 10th December 1991 comes before 11th March 1997, the claim that the term start for Bashkim Fino is after the term start for Vilson Ahmeti is FALSE.

Therefore, the answer is 0.


In [15]:
from dateutil import parser
import datetime
def parse_datetime(date_string):
    parsed_date = parser.parse(date_string)
    if parsed_date is None or not all([parsed_date.year, parsed_date.month, parsed_date.day]):
        return date_string
    print(parsed_date)
    if parsed_date.year == datetime.datetime.now().year:
        normalized_date = datetime.datetime.strftime(parsed_date, "%m-%d")
    else:
        normalized_date = datetime.datetime.strftime(parsed_date, "%Y-%m-%d")
    return normalized_date


In [ ]:
result = [ ]
for i in range(len(table_loader.dataset)):
    sample = table_loader.normalize_table(table_loader.dataset[i])
    llm_chain = LLMChain(llm=model, prompt=stage_0_prompt, verbose=False)
    stage_0_batch_pred = llm_chain.batch([{"query": sample['query']}], return_only_outputs=True)[0]['text'].split(':')[-1]
    print(stage_0_batch_pred)
    sub_queries = stage_0_batch_pred.split(';')

    from langchain_community.callbacks import get_openai_callback
    Agent_history = ChatMessageHistory()
    agent_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Based on the history information, your task is to determine whether the information is enough to answer the user query.
                return 'A' if you think you need more information, else return 'B'. You should only use information during the conversation.""",
            ),
            MessagesPlaceholder(variable_name="chat_history"),
        ]
    )
    chain = LLMChain(llm=model, prompt=agent_prompt, verbose=True)
    with get_openai_callback() as cb:
        for sub_query in sub_queries:
            Agent_history.add_user_message(sub_query)
            choice = chain.invoke(
            {
                "chat_history": Agent_history.messages,
            }
            )['text']
            if 'A' in choice:
                Agent_history.add_ai_message(scene_A(sub_query, sample))
            else:
                res = scene_B()
                print(res)
                result.append(res)
                
    print(cb.total_tokens)
    

In [16]:
import re
import string
import recognizers_suite
from recognizers_text import Culture, ModelResult
def str_normalize(user_input, recognition_types=None):
    """A string normalizer which recognize and normalize value based on recognizers_suite"""
    user_input = str(user_input)
    user_input = user_input.replace("\\n", "; ")

    def replace_by_idx_pairs(orig_str, strs_to_replace, idx_pairs):
        assert len(strs_to_replace) == len(idx_pairs)
        last_end = 0
        to_concat = []
        for idx_pair, str_to_replace in zip(idx_pairs, strs_to_replace):
            to_concat.append(orig_str[last_end : idx_pair[0]])
            to_concat.append(str_to_replace)
            last_end = idx_pair[1]
        to_concat.append(orig_str[last_end:])
        return ''.join(to_concat)

    if recognition_types is None:
        recognition_types = [
            "datetime",
            "number",
            "ordinal",
            "percentage",
            "age",
            "currency",
            "dimension",
            "temperature",
        ]
    culture = Culture.English
    for recognition_type in recognition_types:
        if re.match("\d+/\d+", user_input):
            # avoid calculating str as 1991/92
            continue
        recognized_list = getattr(
            recognizers_suite, "recognize_{}".format(recognition_type)
        )(
            user_input, culture
        )  # may match multiple parts
        strs_to_replace = []
        idx_pairs = []
        for recognized in recognized_list:
            if not recognition_type == 'datetime':
                recognized_value = recognized.resolution['value']
                if str(recognized_value).startswith("P"):
                    # if the datetime is a period:
                    continue
                else:
                    strs_to_replace.append(recognized_value)
                    idx_pairs.append((recognized.start, recognized.end + 1))
            else:
                if recognized.resolution:  # in some cases, this variable could be none.
                    if len(recognized.resolution['values']) == 1:
                        strs_to_replace.append(
                            recognized.resolution['values'][0]['timex']
                        )  # We use timex as normalization
                        idx_pairs.append((recognized.start, recognized.end + 1))

        if len(strs_to_replace) > 0:
            user_input = replace_by_idx_pairs(user_input, strs_to_replace, idx_pairs)

    if re.match("(.*)-(.*)-(.*) 00:00:00", user_input):
        user_input = user_input[: -len("00:00:00") - 1]
        # '2008-04-13 00:00:00' -> '2008-04-13'
    return user_input


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", text)

    def whilt_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return whilt_space_fix(remove_articles(remove_punc(lower(s))))


In [5]:
import string
import re
def normalize_answer(s):
    def remove_articles(text):
        return re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", text)

    def whilt_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return whilt_space_fix(remove_articles(remove_punc(lower(s))))


In [37]:
'cahill' == 'cahill colosimo culina elrich griffiths skoko zdrilic'

False

In [17]:
str_normalize('Dmitry Mikhailovich Golitsyn served longer as an ambassador.')

'Dmitry Mikhailovich Golitsyn served longer as an ambassador.'

In [36]:
def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))
compute_exact('Cahill', 'Cahill, Colosimo, Culina, Elrich, Griffiths, Skoko, Zdrilic')

0

In [27]:
str_normalize(normalize_answer('Sopwith Triplane s/n N5460'))

'sopwith triplane sn n5460'

In [29]:
str_normalize('Cahill, Colosimo, Culina, Elrich, Griffiths, Skoko, Zdrilic')

'Cahill, Colosimo, Culina, Elrich, Griffiths, Skoko, Zdrilic'